# Fifteen minutes walking for urban city planning
To create a 15-minute city, urban planners need to address a variety of place categories to ensure that all essential services and amenities are within easy reach.

In [3]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from arcgis.geometry.functions import project
from geopedestrian.categories import CategoryRegistry
from geopedestrian.services import analyze_walking, solve_walking, walk_from
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from ipywidgets import HBox, Layout

The `host` variable stores the URL for the geopedestrian API hosted on RapidAPI. This URL is used to connect to the GeoPedestrian services.

The `client` variable creates an instance of the `GeoRapidClient` using the `EnvironmentClientFactory`. The `create_client_with_host` method takes the host URL as an argument and returns a client instance for interacting with the geopedestrian API.

In [2]:
host = 'geopedestrian.p.rapidapi.com'
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

This function plots the results of a 15-minute walking analysis using the specified map view. The map view must be fully initialized before calling these functions.

In [30]:
def plot_walking_areas(map_view, client: GeoRapidClient):
    """
    Plots walking areas which are accessible by pedestrians using the specified map view.
    
    :param map_view: The map view defining the area of interest.
    :param client: The authenticated client instance for Rapid API.
    :return: Returns the walking areas as a spatially-enabled dataframe.
    """
    # Project the map view center to WGS84
    spatial_reference = map_view.center['spatialReference']
    in_sr = spatial_reference['wkid']
    out_sr = 4326
    project_result = project([map_view.center], in_sr, out_sr)
    location = project_result[0]
    lat, lon = location.y, location.x
    
    # Solve the walking areas
    walking_result = solve_walking(client, lat, lon)
    walking_areas_sdf = FeatureSet.from_geojson(walking_result).sdf
    walking_areas_sdf.spatial.plot(map_view, renderer_type='u', col='ToBreak', cmap='RdYlGn', alpha=0.35)
    map_view.zoom_to_layer(walking_areas_sdf)
    return walking_areas_sdf

def plot_walking_routes(map_view, client: GeoRapidClient, category_id: int):
    """
    Plots walking routes to places which are accessible by pedestrians using the specified map view.
    
    :param map_view: The map view defining the area of interest.
    :param client: The authenticated client instance for Rapid API.
    :param category_id: The places category of interest.
    :return: Returns the walking routes as a spatially-enabled dataframe.
    """
    # Project the map view center to WGS84
    spatial_reference = map_view.center['spatialReference']
    in_sr = spatial_reference['wkid']
    out_sr = 4326
    project_result = project([map_view.center], in_sr, out_sr)
    location = project_result[0]
    lat, lon = location.y, location.x
    
    # Solve the walking routes
    walking_result = walk_from(client, lat, lon, category_id)
    routes_geojson = walking_result['routes']
    routes_sdf = FeatureSet.from_geojson(routes_geojson).sdf
    routes_sdf.spatial.plot(map_view, renderer_type='s', palette=[0, 120, 0, 130], line_width=15)
    map_view.zoom_to_layer(routes_sdf)
    return routes_sdf

def plot_places_nearby(map_view, client: GeoRapidClient, category_id: int):
    """
    Plots places which are accessible by pedestrians using the specified map view.
    
    :param map_view: The map view defining the area of interest.
    :param client: The authenticated client instance for Rapid API.
    :param category_id: The places category of interest.
    :return: Returns the places as a spatially-enabled dataframe.
    """
    # Project the map view center to WGS84
    spatial_reference = map_view.center['spatialReference']
    in_sr = spatial_reference['wkid']
    out_sr = 4326
    project_result = project([map_view.center], in_sr, out_sr)
    location = project_result[0]
    lat, lon = location.y, location.x
    
    # Solve the walking routes
    walking_result = walk_from(client, lat, lon, category_id)
    places_geojson = walking_result['places']
    places_sdf = FeatureSet.from_geojson(places_geojson).sdf
    places_sdf.spatial.plot(map_view)
    map_view.zoom_to_layer(places_sdf)
    return places_sdf

def plot_urban_walking(map_view, client: GeoRapidClient, category_id: int):
    """
    Plots the 15 minutes walking analysis result using the specified map view.
    The map view must be fully initialized.
    
    :param map_view: The map view defining the area of interest.
    :param client: The authenticated client instance for Rapid API.
    :param category_id: The places category of interest.
    :return: Returns the analysis result as a spatially-enabled dataframe.
    """
    # Project the map view center to WGS84
    spatial_reference = map_view.center['spatialReference']
    in_sr = spatial_reference['wkid']
    out_sr = 4326
    project_result = project([map_view.center], in_sr, out_sr)
    location = project_result[0]
    lat, lon = location.y, location.x
    
    # Analyze the 15 minutes walking areas
    analysis_result = analyze_walking(client, lat, lon, category_id)
    analysis_sdf = FeatureSet.from_geojson(analysis_result).sdf
    analysis_sdf.spatial.plot(map_view, renderer_type='u', col='hits', cmap='YlGn', alpha=0.35)
    map_view.zoom_to_layer(analysis_sdf)
    return analysis_sdf

## Define the categories of interest
This line of code creates an instance of the `CategoryRegistry` class and assigns it to the `category_registry` variable. The `CategoryRegistry` instance can be used to manage and access different categories of places for the 15-minute city analysis.

In [6]:
category_registry = CategoryRegistry()

This line of code searches the `CategoryRegistry` instance for the category ID corresponding to the category name "Museum". The resulting ID is then assigned to the museum_id variable. This ID can be used in subsequent analysis or API calls to identify places related to museums.

In [7]:
museum_id = category_registry.find_id('Museum')
school_id = category_registry.find_id('Primary and Secondary School')
university_id = category_registry.find_id('College and University')
restaurant_id = category_registry.find_id('Restaurant')
cafe_id = category_registry.find_id('Cafe')
bar_id = category_registry.find_id('Bar')
park_id = category_registry.find_id('Park')
sports_id = category_registry.find_id('Sports Club')
residential_id = category_registry.find_id('Residential Building')
office_id = category_registry.find_id('Office')
retail_id = category_registry.find_id('Retail')
transport_id = category_registry.find_id('Transport Hub')

## Visualize the 15 minutes walking scenarios for various cities

In [8]:
gis = GIS()

In [9]:
def create_map_view(gis, named_location):
    map_view = gis.map(named_location)
    map_view.basemap = 'osm'
    return map_view

def display_two_map_views(left_map_view, right_map_view):
    left_map_view.layout=Layout(flex='1 1', padding='6px', height='450px')
    right_map_view.layout=Layout(flex='1 1', padding='6px', height='450px')

    return HBox([left_map_view, right_map_view])

In [13]:
paris_map = create_map_view(gis, 'Paris, France')
paris_map

MapView(layout=Layout(height='400px', width='100%'))

In [17]:
plot_walking_areas(paris_map, client)

In [29]:
paris_map = create_map_view(gis, 'Paris, France')
paris_map

MapView(layout=Layout(height='400px', width='100%'))

In [31]:
plot_places_nearby(paris_map, client, museum_id)

,placeId,name,from,to,OBJECTID,SHAPE
0,d5f202cbf3e1e1d3261b394ab7982336,Musée de l'Histoire de France,5,10,1,"{""x"": 2.357099, ""y"": 48.860024, ""spatialRefere..."
1,3028d0c324e3b4fb83f40d9e42d5a836,Musée Picasso,5,10,2,"{""x"": 2.362286, ""y"": 48.859905, ""spatialRefere..."
2,e572d38397e895e9832118b3eafb61ba,Musée de la Chasse et de la Nature,0,5,3,"{""x"": 2.358624, ""y"": 48.861507, ""spatialRefere..."
3,e69c4813788b76e802e640c7b3dc9ae1,Fondation Henri Cartier-Bresson,0,5,4,"{""x"": 2.359913, ""y"": 48.863244, ""spatialRefere..."
4,c2ab5c711d7591387317006c98d749f1,Passage de Retz,0,5,5,"{""x"": 2.360545, ""y"": 48.861308, ""spatialRefere..."


In [22]:
paris_map = create_map_view(gis, 'Paris, France')
paris_map

MapView(layout=Layout(height='400px', width='100%'))

In [25]:
plot_walking_routes(paris_map, client, museum_id)

,name,total_minutes,total_kilometers,OBJECTID,SHAPE
0,Fondation Henri Cartier-Bresson - Location 1,2.19,0.18,1,"{""paths"": [[[2.360091761000035, 48.86358195500..."
1,Passage de Retz - Location 1,4.27,0.36,2,"{""paths"": [[[2.3605823910000368, 48.8613948180..."
2,Musée de la Chasse et de la Nature - Location 1,4.56,0.38,3,"{""paths"": [[[2.3585921190000363, 48.8615440690..."
3,Musée Picasso - Location 1,6.53,0.54,4,"{""paths"": [[[2.362382298000057, 48.86006192600..."
4,Musée de l'Histoire de France - Location 1,7.25,0.6,5,"{""paths"": [[[2.357057196000028, 48.85996773100..."


In [26]:
paris_map = create_map_view(gis, 'Paris, France')
paris_map

MapView(layout=Layout(height='400px', width='100%'))

In [27]:
plot_urban_walking(paris_map, client, museum_id)

,hits,OBJECTID,SHAPE
0,1,1,"{""rings"": [[[2.3549608912799354, 48.8545387117..."
1,1,2,"{""rings"": [[[2.3549608912799354, 48.8607445322..."
2,1,3,"{""rings"": [[[2.3549608912799354, 48.8660631946..."
3,1,4,"{""rings"": [[[2.3561278370649186, 48.8603012848..."
4,1,5,"{""rings"": [[[2.3607956202048506, 48.8567551645..."
5,1,6,"{""rings"": [[[2.3607956202048506, 48.8585282561..."
6,1,7,"{""rings"": [[[2.3619625659898333, 48.8598580335..."
7,1,8,"{""rings"": [[[2.3666303491297653, 48.8589715192..."


In [37]:
barcelona_map = create_map_view(gis, 'Barcelona, Spain')
barcelona_map

MapView(layout=Layout(height='400px', width='100%'))

In [38]:
plot_urban_walking(barcelona_map, client, museum_id)

,hits,OBJECTID,SHAPE
0,1,1,"{""rings"": [[[2.15837150178505, 41.395660534716..."
1,1,2,"{""rings"": [[[2.1607053933550167, 41.3956605347..."
2,1,3,"{""rings"": [[[2.1642062307099654, 41.3931334309..."
3,1,4,"{""rings"": [[[2.1700409596348806, 41.3804964386..."
4,1,5,"{""rings"": [[[2.1723748512048466, 41.3825185224..."
5,1,6,"{""rings"": [[[2.175875688559796, 41.38201300738..."
6,1,7,"{""rings"": [[[2.175875688559796, 41.38403504405..."
7,1,8,"{""rings"": [[[2.1770426343447786, 41.3835295407..."
8,1,9,"{""rings"": [[[2.1782095801297614, 41.3810019654..."


In [39]:
display_two_map_views(paris_map, barcelona_map)